In [3]:
import pandas as pd
import matplotlib.pyplot as plt

df_hits = pd.read_csv("additional/hits_dataset.csv", delimiter="\t")
df_hits["hit"] = 1
df_hits

,song_id,song_name,id_artists,name_artists,popularity,explicit,song_type,track_number,num_artists,num_available_markets,...,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,valence,tempo,hit
0,3e9HZxeyfWwjeyPAMmWSSQ,"thank u, next",['66CXWjxzNUsdJxJ2JdwvnR'],['Ariana Grande'],87,True,Solo,11,1,79,...,0.22900,0.717,0.653,0.000000,0.1010,-5.634,0.0658,0.412,106.966,1
1,5p7ujcrUXASCNwRaWNHR1C,Without Me,['26VFTg2z8YR0cCuwLzESi2'],['Halsey'],87,True,Solo,1,1,79,...,0.29700,0.752,0.488,0.000009,0.0936,-7.050,0.0705,0.533,136.041,1
2,2xLMifQCjDGFmkHkpNLD9h,SICKO MODE,['0Y5tJX1MQlPlqiwlOH1tJY'],['Travis Scott'],86,True,Solo,3,1,79,...,0.00513,0.834,0.730,0.000000,0.1240,-3.714,0.2220,0.446,155.008,1
3,3KkXRkHbMCARz0aVfEt68P,Sunflower - Spider-Man: Into the Spider-Verse,"['246dkjvS1zLTtiykXe5h60', '1zNqQNIdeOUZHb8zbZ...","['Post Malone', 'Swae Lee']",92,False,Collaboration,2,2,79,...,0.55600,0.760,0.479,0.000000,0.0703,-5.574,0.0466,0.913,89.911,1
4,1rqqCSm0Qe4I9rUvWncaom,High Hopes,['20JZFwl6HVl6yg8a4H3ZqK'],['Panic! At The Disco'],86,False,Solo,4,1,78,...,0.19300,0.579,0.904,0.000000,0.0640,-2.729,0.0618,0.681,82.014,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11954,5mz9pQZZXNpAw9CdQ7Bk8q,Don't Pity Me - Mono; 2003 Remaster,['6lHC2EQMEMZiEmSfFloarn'],['Peter And Gordon'],8,False,Solo,1,1,78,...,0.61900,0.500,0.654,0.000000,0.3320,-8.633,0.0394,0.403,107.326,1
11955,2H9CKpZiLDF223BbwehpDF,Hush Hush; Hush Hush - Main,['6wPhSqRtPu1UhRCDX5yaDJ'],['The Pussycat Dolls'],57,False,Solo,1,1,59,...,0.10500,0.609,0.790,0.000000,0.3140,-3.787,0.0663,0.664,129.334,1
11956,7zqM7Wm2xJLZ50qAnTLoHD,"Lions, Tigers & Bears",['7gSjFKpVmDgC2MMsnN8CYq'],['Jazmine Sullivan'],49,False,Solo,4,1,71,...,0.07880,0.576,0.515,0.000000,0.1180,-6.178,0.0295,0.407,95.406,1
11957,4DPlKVZ6R1EHLfnIsy9ouh,The Weight,"['0rXI0q8Cahq6numvPlloaq', '3RwQ26hR2tJtA8F9p2...","['Diana Ross & The Supremes', 'The Temptations']",2,False,Collaboration,23,2,79,...,0.37000,0.566,0.607,0.000000,0.6710,-7.532,0.0357,0.765,91.378,1


In [4]:
df_nonhits = pd.read_csv("additional/nonhits_dataset.csv", delimiter="\t")
df_nonhits["hit"] = 0
df_nonhits

C:\Users\lasse\AppData\Local\Temp\ipykernel_25812\2533836516.py:1: DtypeWarning: Columns (11,12,13,14,15,16,17,18,19,20,21,22,23) have mixed types. Specify dtype option on import or set low_memory=False.
  df_nonhits = pd.read_csv("additional/nonhits_dataset.csv", delimiter="\t")


,song_id,song_name,id_artists,name_artists,popularity,explicit,song_type,track_number,num_artists,num_available_markets,...,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,valence,tempo,hit
0,1TEL6MlSSVLSdhOSddidlJ,needy,['66CXWjxzNUsdJxJ2JdwvnR'],['Ariana Grande'],76,True,Solo,2,1,79,...,0.78,0.647,0.309,7.41e-06,0.202,-7.948,0.0366,0.195,87.045,0
1,4uTvPEr01pjTbZgl7jcKBD,NASA,['66CXWjxzNUsdJxJ2JdwvnR'],['Ariana Grande'],76,False,Solo,3,1,79,...,0.451,0.747,0.458,0,0.252,-6.891,0.303,0.47,75.029,0
2,2hloaUoRonYssMuqLCBLTX,bloodline,['66CXWjxzNUsdJxJ2JdwvnR'],['Ariana Grande'],74,True,Solo,4,1,79,...,0.0815,0.758,0.665,0.000157,0.216,-5.188,0.0774,0.643,151.976,0
3,3wFLWP0FcIqHK1wb1CPthQ,fake smile,['66CXWjxzNUsdJxJ2JdwvnR'],['Ariana Grande'],73,True,Solo,5,1,79,...,0.329,0.45,0.619,0,0.319,-7.412,0.372,0.427,83.3,0
4,5Il6Oe7lr5XM7A0cWbVQtr,bad idea,['66CXWjxzNUsdJxJ2JdwvnR'],['Ariana Grande'],74,False,Solo,6,1,79,...,0.0268,0.847,0.579,0.0545,0.102,-5.314,0.0719,0.569,137.998,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
899063,2tP0ryZParpdjvxubgXhTN,Love Is Here to Stay,"['3Dzj993UEz8Z5ovxuirzFO', '6RCsx4p5smZHYz2P5H...","['Lindsey Buckingham', 'Christine McVie']",16,False,Collaboration,5,2,1,...,0.366,0.446,0.658,2.49e-05,0.751,-8.362,0.0297,0.554,84.74,0
899064,7Ltni80U0w7bmYjXPNzSEa,Heinlein's Planet,['4urvwbid7NFoVes5PTx9lV'],['Tony Carey'],0,False,Solo,5,1,79,...,0.481,0.305,0.456,0.933,0.111,-20.199,0.0634,0.171,96.996,0
899065,1gQClXN1Dq4rj0wBzddcNA,Beds are Burning - Karaoke Lead Vocal Version,['7yv4DJOCmilSbxxsdQgXWk'],['Pictomusic'],0,False,Solo,2,1,79,...,0.0118,0.852,0.648,0,0.0503,-7.189,0.048,0.559,120.15,0
899066,2jhxQrFNdVPkWL28X7ZpeK,Un homme heureux - Version karaoké instrumentale,['7yv4DJOCmilSbxxsdQgXWk'],['Pictomusic'],0,False,Solo,1,1,79,...,0.973,0.664,0.353,0.787,0.0878,-12.303,0.0693,0.284,116.45,0


In [5]:
combined_df = pd.concat([df_hits, df_nonhits], axis=0)
combined_df.replace("-", float("nan"), inplace=True)
# Drop all rows with NaN values
combined_df

,song_id,song_name,id_artists,name_artists,popularity,explicit,song_type,track_number,num_artists,num_available_markets,...,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,valence,tempo,hit
0,3e9HZxeyfWwjeyPAMmWSSQ,"thank u, next",['66CXWjxzNUsdJxJ2JdwvnR'],['Ariana Grande'],87,True,Solo,11,1,79,...,0.229,0.717,0.653,0.0,0.101,-5.634,0.0658,0.412,106.966,1
1,5p7ujcrUXASCNwRaWNHR1C,Without Me,['26VFTg2z8YR0cCuwLzESi2'],['Halsey'],87,True,Solo,1,1,79,...,0.297,0.752,0.488,0.000009,0.0936,-7.05,0.0705,0.533,136.041,1
2,2xLMifQCjDGFmkHkpNLD9h,SICKO MODE,['0Y5tJX1MQlPlqiwlOH1tJY'],['Travis Scott'],86,True,Solo,3,1,79,...,0.00513,0.834,0.73,0.0,0.124,-3.714,0.222,0.446,155.008,1
3,3KkXRkHbMCARz0aVfEt68P,Sunflower - Spider-Man: Into the Spider-Verse,"['246dkjvS1zLTtiykXe5h60', '1zNqQNIdeOUZHb8zbZ...","['Post Malone', 'Swae Lee']",92,False,Collaboration,2,2,79,...,0.556,0.76,0.479,0.0,0.0703,-5.574,0.0466,0.913,89.911,1
4,1rqqCSm0Qe4I9rUvWncaom,High Hopes,['20JZFwl6HVl6yg8a4H3ZqK'],['Panic! At The Disco'],86,False,Solo,4,1,78,...,0.193,0.579,0.904,0.0,0.064,-2.729,0.0618,0.681,82.014,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
899063,2tP0ryZParpdjvxubgXhTN,Love Is Here to Stay,"['3Dzj993UEz8Z5ovxuirzFO', '6RCsx4p5smZHYz2P5H...","['Lindsey Buckingham', 'Christine McVie']",16,False,Collaboration,5,2,1,...,0.366,0.446,0.658,2.49e-05,0.751,-8.362,0.0297,0.554,84.74,0
899064,7Ltni80U0w7bmYjXPNzSEa,Heinlein's Planet,['4urvwbid7NFoVes5PTx9lV'],['Tony Carey'],0,False,Solo,5,1,79,...,0.481,0.305,0.456,0.933,0.111,-20.199,0.0634,0.171,96.996,0
899065,1gQClXN1Dq4rj0wBzddcNA,Beds are Burning - Karaoke Lead Vocal Version,['7yv4DJOCmilSbxxsdQgXWk'],['Pictomusic'],0,False,Solo,2,1,79,...,0.0118,0.852,0.648,0,0.0503,-7.189,0.048,0.559,120.15,0
899066,2jhxQrFNdVPkWL28X7ZpeK,Un homme heureux - Version karaoké instrumentale,['7yv4DJOCmilSbxxsdQgXWk'],['Pictomusic'],0,False,Solo,1,1,79,...,0.973,0.664,0.353,0.787,0.0878,-12.303,0.0693,0.284,116.45,0


In [6]:
# Check for NaN values in all columns
nan_columns = combined_df.columns[combined_df.isna().any()].tolist()

# Print columns with NaN values
print("Columns with NaN values:")
print(nan_columns)


Columns with NaN values:
['duration_ms', 'key', 'mode', 'time_signature', 'acousticness', 'danceability', 'energy', 'instrumentalness', 'liveness', 'loudness', 'speechiness', 'valence', 'tempo']


In [7]:
from sklearn.impute import SimpleImputer

combined_df.replace("-", float("nan"), inplace=True)   ## evtl hier schon mean replacement???
# combined_df.dropna(inplace=True)   # drop nan value rows
#Fill missing values with mean for each numeric attribute
imputer = SimpleImputer(strategy='mean')
for col in nan_columns:
    combined_df[col] = imputer.fit_transform(combined_df[[col]])

combined_df

,song_id,song_name,id_artists,name_artists,popularity,explicit,song_type,track_number,num_artists,num_available_markets,...,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,valence,tempo,hit
0,3e9HZxeyfWwjeyPAMmWSSQ,"thank u, next",['66CXWjxzNUsdJxJ2JdwvnR'],['Ariana Grande'],87,True,Solo,11,1,79,...,0.22900,0.717,0.653,0.000000,0.1010,-5.634,0.0658,0.412,106.966,1
1,5p7ujcrUXASCNwRaWNHR1C,Without Me,['26VFTg2z8YR0cCuwLzESi2'],['Halsey'],87,True,Solo,1,1,79,...,0.29700,0.752,0.488,0.000009,0.0936,-7.050,0.0705,0.533,136.041,1
2,2xLMifQCjDGFmkHkpNLD9h,SICKO MODE,['0Y5tJX1MQlPlqiwlOH1tJY'],['Travis Scott'],86,True,Solo,3,1,79,...,0.00513,0.834,0.730,0.000000,0.1240,-3.714,0.2220,0.446,155.008,1
3,3KkXRkHbMCARz0aVfEt68P,Sunflower - Spider-Man: Into the Spider-Verse,"['246dkjvS1zLTtiykXe5h60', '1zNqQNIdeOUZHb8zbZ...","['Post Malone', 'Swae Lee']",92,False,Collaboration,2,2,79,...,0.55600,0.760,0.479,0.000000,0.0703,-5.574,0.0466,0.913,89.911,1
4,1rqqCSm0Qe4I9rUvWncaom,High Hopes,['20JZFwl6HVl6yg8a4H3ZqK'],['Panic! At The Disco'],86,False,Solo,4,1,78,...,0.19300,0.579,0.904,0.000000,0.0640,-2.729,0.0618,0.681,82.014,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
899063,2tP0ryZParpdjvxubgXhTN,Love Is Here to Stay,"['3Dzj993UEz8Z5ovxuirzFO', '6RCsx4p5smZHYz2P5H...","['Lindsey Buckingham', 'Christine McVie']",16,False,Collaboration,5,2,1,...,0.36600,0.446,0.658,0.000025,0.7510,-8.362,0.0297,0.554,84.740,0
899064,7Ltni80U0w7bmYjXPNzSEa,Heinlein's Planet,['4urvwbid7NFoVes5PTx9lV'],['Tony Carey'],0,False,Solo,5,1,79,...,0.48100,0.305,0.456,0.933000,0.1110,-20.199,0.0634,0.171,96.996,0
899065,1gQClXN1Dq4rj0wBzddcNA,Beds are Burning - Karaoke Lead Vocal Version,['7yv4DJOCmilSbxxsdQgXWk'],['Pictomusic'],0,False,Solo,2,1,79,...,0.01180,0.852,0.648,0.000000,0.0503,-7.189,0.0480,0.559,120.150,0
899066,2jhxQrFNdVPkWL28X7ZpeK,Un homme heureux - Version karaoké instrumentale,['7yv4DJOCmilSbxxsdQgXWk'],['Pictomusic'],0,False,Solo,1,1,79,...,0.97300,0.664,0.353,0.787000,0.0878,-12.303,0.0693,0.284,116.450,0


In [8]:
# Check for NaN values in all columns
nan_columns = combined_df.columns[combined_df.isna().any()].tolist()

# Print columns with NaN values
print("Columns with NaN values:")
print(nan_columns)


Columns with NaN values:
[]


In [9]:
from sklearn.model_selection import train_test_split
combined_df['date'] = pd.to_datetime(combined_df['release_date'])
combined_df.sort_values(by="date", inplace=True)
# border_day = combined_df["date"].iloc[-1]  - pd.DateOffset(years=6)
# combined_df = combined_df[(combined_df["date"] >= border_day)]
# 
# split_day = combined_df["date"].iloc[-1]  - pd.DateOffset(years=1)
# train_df = combined_df[(combined_df["date"]  < split_day)]
# test_df = combined_df[(combined_df["date"]  >= split_day)]
train_df, test_df = train_test_split(combined_df, test_size=1/4, shuffle=False)

In [10]:
combined_df

,song_id,song_name,id_artists,name_artists,popularity,explicit,song_type,track_number,num_artists,num_available_markets,...,danceability,energy,instrumentalness,liveness,loudness,speechiness,valence,tempo,hit,date
804676,7kXUEJmfvRXbzxOC0pHQgb,I Can't Believe I'm Losing You,['6eVCRmRG2HcfL4yx77A9WK'],['Jack Jones'],5,False,Solo,12,1,79,...,0.180,0.369,0.000000,0.1500,-9.797,0.0299,0.224,81.701,0,1995-01-01
465447,1xdRFXv3ytSOdj8GuT3Rge,Thank You Lord (Interlude),['5NDMothbpdpq2xHqSjrrWn'],['Faith Evans'],20,False,Solo,10,1,77,...,0.370,0.284,0.000000,0.5580,-14.443,0.0378,0.649,130.985,0,1995-01-01
465446,5wlQIBbKIWoB7AnSw09C6C,All This Love,['5NDMothbpdpq2xHqSjrrWn'],['Faith Evans'],26,False,Solo,9,1,77,...,0.605,0.604,0.000153,0.0693,-9.214,0.0480,0.503,145.483,0,1995-01-01
377956,40uHAEjua1YmyKIJ6k0iVa,Bo Bo Wa,['3z6Uf5IZkN4TogRpRpgD84'],['Gary Wright'],0,False,Solo,8,1,79,...,0.481,0.652,0.008670,0.0397,-10.981,0.0364,0.795,195.791,0,1995-01-01
465445,2zmMWWgwCXYarkO2SbyPZP,Come Over,['5NDMothbpdpq2xHqSjrrWn'],['Faith Evans'],38,False,Solo,7,1,77,...,0.534,0.563,0.000001,0.5740,-8.975,0.0601,0.414,124.149,0,1995-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11098,0xAqhyWvTkfcdFgm7dozMO,(I Wanna) Testify (Mono Single Version),['1Ve5PoI7i5jIIP1xudo6AB'],['The Parliaments'],7,False,Solo,10,1,79,...,0.668,0.702,0.000000,0.0521,-5.750,0.0330,0.754,105.227,1,2019-08-23
10903,6zhg8TnVKi90ITDzFKwPr0,Swingtown - Alternate Version,['6QtGlUje9TIkLrgPZrESuk'],['Steve Miller Band'],26,False,Solo,5,1,79,...,0.459,0.764,0.000544,0.2440,-6.662,0.0354,0.439,124.557,1,2019-08-23
5856,2GvkFSjRXV0Ii3uYnHiQ74,Take My Advice,['40PLSl99xXqAQsCQQPmfyM'],['Kym Sims'],0,False,Solo,10,1,73,...,0.737,0.814,0.392000,0.0844,-10.396,0.0307,0.803,120.010,1,2019-08-23
5080,0lej8CemYKkWVHg77Xe1ky,A Man Holdin' on (To a Woman Lettin’ Go),"['1Fj0R2t4HaJa3oUe8azB8R', '5LDnSV99gbDSsMX3Dc...","['Ty Herndon', 'Anita Cochran']",14,False,Collaboration,7,2,79,...,0.611,0.429,0.000014,0.1120,-8.619,0.0298,0.237,128.007,1,2019-08-23


In [11]:
# Specify the columns
columns = ['artist_id', 'artist_name']

# Initialize an empty DataFrame
df_artists_train = pd.DataFrame(columns={col: [] for col in columns})

count = 0
artist_set = set()

for _, row in train_df.iterrows():
    current_artist_id_list = eval(row["id_artists"])
    current_artist_name_list = eval(row["name_artists"])
    for cur_art_id, cur_art_name in zip(current_artist_id_list, current_artist_name_list):
        if cur_art_id not in artist_set:
            artist_set.add(cur_art_id)
            df_artists_train.loc[len(df_artists_train)] = [cur_art_id, cur_art_name]
            count += 1

print("amount of artists:", count)
df_artists_train

amount of artists: 18551


,artist_id,artist_name
0,6eVCRmRG2HcfL4yx77A9WK,Jack Jones
1,5NDMothbpdpq2xHqSjrrWn,Faith Evans
2,3z6Uf5IZkN4TogRpRpgD84,Gary Wright
3,2UuKUaomk3bZ0nHM3Y3FdW,Bud Shank
4,6S0GHTqz5sxK5f9HtLXn9q,Garbage
...,...,...
18546,3bj9NXSjQfajNgU9woBnrR,Country Pop All-Stars
18547,7ueug5uU2UKBoGQInMsKQV,Deep Noise
18548,4UD3hwaHMCoJ9GKgIoMzTp,Steve Dafoe
18549,2OdSf9rTLg6GYfoOrK2Kjl,Ron Komie


In [12]:
import networkx as nx

all_artist_ids = artist_set.copy()

G = nx.Graph()

for id_art_cur in all_artist_ids:
    artist_name = df_artists_train[(df_artists_train["artist_id"] == id_art_cur)]["artist_name"]
    artist_info = df_artists_train[df_artists_train["artist_id"] == id_art_cur]
    if not artist_info.empty:  # Check if artist info exists
        # Filter out artists who are not successful
        node_attrs = {"name": artist_name.iloc[0]}
        G.add_node(id_art_cur, **node_attrs)
    else:
        print("Artist info not found for ID:", id_art_cur)

for _, row in train_df.iterrows():
    artist_ids = list(eval(row["id_artists"]))
    artist_names = list(eval(row["name_artists"]))
    # Filter out songs that do not have more than one artist in their execution
    if len(artist_ids) > 1:
        for i in range(len(artist_ids)):
            for j in range(i+1, len(artist_ids)):
                artist_id_1 = str(artist_ids[i])
                lable1 = str(artist_names[i])
                artist_id_2 = str(artist_ids[j])
                lable2 = str(artist_names[j])
                # Check if the edge already exists
                if G.has_edge(artist_id_1, artist_id_2):
                    # If the edge already exists, increment the weight by 1
                    G[artist_id_1][artist_id_2]['weight'] += 2
                else:
                    # Add a new edge with weight 1
                    G.add_edge(artist_id_1, artist_id_2, lable1=lable1, lable2=lable2, weight=2, song_id=str(row["song_id"]), song_name=str(row["song_name"]), explicit=bool(row["explicit"]), song_type=str(row["song_type"]), track_number=int(row["track_number"]), num_artists=int(row["num_artists"]), num_available_markets=int(row["num_available_markets"]), duration_ms=int(row["duration_ms"]), popularity=int(row["popularity"]), release_date=str(row["release_date"]), key = int(row["key"]), mode = int(row["mode"]), time_signature = int(row["time_signature"]), acousticness = float(row["acousticness"]), danceability = float(row["danceability"]), energy = float(row["energy"]), instrumentalness = float(row["instrumentalness"]), liveness = float(row["liveness"]),loudness = float(row["loudness"]), speechiness = float(row["speechiness"]), valence = float(row["valence"]), tempo = float(row["tempo"]))
    else:
        #prüfe ob die Artists Teil der Interessanten Menge sind
        artist_id_1 = str(artist_ids[0])
        lable1 = str(artist_names[0])
        if not G.has_node(artist_id_1):
            node_attrs = {"name": lable1}
            G.add_node(artist_id_1, **node_attrs)
        if G.has_edge(artist_id_1, artist_id_1):
            # If the edge already exists, increment the weight by 1
            G[artist_id_1][artist_id_1]['weight'] += 2
        else:
            G.add_edge(artist_id_1, artist_id_1, lable1=lable1, lable2=lable1, weight=2, song_id=str(row["song_id"]), song_name=str(row["song_name"]), explicit=bool(row["explicit"]), song_type=str(row["song_type"]), track_number=int(row["track_number"]), num_artists=int(row["num_artists"]), num_available_markets=int(row["num_available_markets"]), duration_ms=int(row["duration_ms"]), popularity=int(row["popularity"]), release_date=str(row["release_date"]), key = int(row["key"]), mode = int(row["mode"]), time_signature = int(row["time_signature"]), acousticness = float(row["acousticness"]), danceability = float(row["danceability"]), energy = float(row["energy"]), instrumentalness = float(row["instrumentalness"]), liveness = float(row["liveness"]),loudness = float(row["loudness"]), speechiness = float(row["speechiness"]), valence = float(row["valence"]), tempo = float(row["tempo"]))

In [13]:
# Assuming G is your NetworkX graph

# Extract nodes and their attributes from the graph
nodes_with_attributes = [(G.nodes[node], node) for node in G.nodes()]

# Create a DataFrame from the nodes and their attributes
df_nodes_train = pd.DataFrame(nodes_with_attributes, columns=['Attributes', "Spotify ID"])

# Normalize the 'Attributes' column to expand it into separate columns
df_attributes = pd.json_normalize(df_nodes_train['Attributes'])

# Combine the original 'Node' column with the expanded attributes
df_nodes_train = pd.concat([df_attributes, df_nodes_train['Spotify ID']], axis=1)

# Display the DataFrame
df_nodes_train

,name,Spotify ID
0,Luca M,6H6VFCKcJLUKANeWVr5mYU
1,O.D.M.,2CytztStk9LFhlOqWiz6Lr
2,Napoleon,3rUqaKQuOgNiClxsadaqD5
3,Dj Derezon,5N76quYhlgDJNMfLl4DizG
4,Verónica Castro,3kuWrRbgBw1HEMmv61DhRr
...,...,...
18546,Carroll Caza,7G9kMNh7rbpqNwMVOlMFd5
18547,Mike Triay,5yocotBPM6JjNb7lvdI0t9
18548,Toto,0PFtn5NtBbbUNbU9EAmIWF
18549,Bill Cosby,4JeqRr8Upw5uxLEu6jgIRm


In [14]:
df_nodes_train.to_csv("network_created/nodes_real_train.csv", sep='\t', index=False)

In [15]:
# Extract edges and their attributes from the graph
edges_with_attributes = [(u, v, G.edges[u, v]) for u, v in G.edges()]

# Create a DataFrame from the edges and their attributes
df_edges_train = pd.DataFrame(edges_with_attributes, columns=['Source', 'Target', 'Attributes'])

# Normalize the 'Attributes' column to expand it into separate columns
df_edge_attributes = pd.json_normalize(df_edges_train['Attributes'])

# Combine the original 'Attributes' column with the expanded attributes
df_edges_train = pd.concat([df_edges_train[['Source', 'Target']], df_edge_attributes], axis=1)
df_edges_train

,Source,Target,lable1,lable2,weight,song_id,song_name,explicit,song_type,track_number,...,time_signature,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,valence,tempo
0,6H6VFCKcJLUKANeWVr5mYU,6H6VFCKcJLUKANeWVr5mYU,Luca M,Luca M,2,5k3FqlbkZ9h0EehE2yTifn,Stuky - Original Mix,False,Solo,9,...,4,0.02710,0.824,0.654,0.935000,0.0866,-11.590,0.0874,0.429,127.019
1,2CytztStk9LFhlOqWiz6Lr,7zgtJuRm0JcJpPQRjCgXT7,N2DEEP,O.D.M.,2,1Aw5hKkwb9tV0jspO8MCl4,Summertime In The City,False,Collaboration,11,...,4,0.05740,0.856,0.677,0.000064,0.0872,-5.082,0.1690,0.766,97.028
2,3rUqaKQuOgNiClxsadaqD5,4gwpcMTbLWtBUlOijbVpuu,Capital Cities,Napoleon,2,2Ab6PMYtYsZkdZ4M0ciA0V,Patience Gets Us Nowhere Fast - Napoleon Remix,False,Collaboration,14,...,4,0.31800,0.670,0.517,0.000765,0.1080,-9.944,0.0297,0.451,128.014
3,5N76quYhlgDJNMfLl4DizG,6CsOL9C9iLL0eWKINlDxzQ,Dj Derezon,Jeru The Damaja,6,6h9bMnDhxu0M02Bwz3a7Sf,Real Rap - Radio Edit,False,Collaboration,1,...,4,0.00551,0.534,0.848,0.000166,0.0454,-7.217,0.1900,0.727,100.005
4,3kuWrRbgBw1HEMmv61DhRr,2AZOALDIBORfbzKTuliwdJ,Cristian Castro,Verónica Castro,2,0qnaHm6dRj7lANM5UU3fXi,Medley: Aprendí A Llorar/Ven - Primera Fila - ...,False,Collaboration,9,...,4,0.02460,0.513,0.638,0.000000,0.9070,-6.741,0.0351,0.458,160.069
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26166,0LthCv77K8NqLKr8B6266a,0LthCv77K8NqLKr8B6266a,Freak Nasty,Freak Nasty,366,5ELn91KxtbHoJhi7wdKEv1,Rumors Pt. 2,True,Solo,18,...,4,0.74300,0.649,0.231,0.000000,0.1400,-24.100,0.9310,0.581,100.066
26167,7G9kMNh7rbpqNwMVOlMFd5,7G9kMNh7rbpqNwMVOlMFd5,Carroll Caza,Carroll Caza,4,2smSnPpqfWeNEOajlegB0k,Dolce Veneno,False,Solo,5,...,3,0.91100,0.182,0.294,0.000736,0.1650,-9.552,0.0294,0.129,164.500
26168,0PFtn5NtBbbUNbU9EAmIWF,0PFtn5NtBbbUNbU9EAmIWF,Toto,Toto,382,7KsFuGDR0lDHS02Xw5Kk72,Blackeye,False,Solo,13,...,4,0.00276,0.656,0.621,0.001210,0.2920,-6.922,0.0260,0.775,76.519
26169,4JeqRr8Upw5uxLEu6jgIRm,4JeqRr8Upw5uxLEu6jgIRm,Bill Cosby,Bill Cosby,14,0ImUvukVkCM6G4FoApWuKG,"Fat Albert's Car - Live At Harrah's Hotel, Reno",False,Solo,1,...,3,0.92400,0.457,0.409,0.000018,0.7830,-16.835,0.9370,0.363,65.939


In [16]:
# Save edge DataFrame to CSV
df_edges_train.to_csv('network_created/edges_real_train.csv', sep='\t', index=False)

In [17]:
num_nodes = G.number_of_nodes()
num_edges = G.number_of_edges()
print("Number of nodes:", num_nodes)
print("Number of edges:", num_edges)


Number of nodes: 18551
Number of edges: 26171


In [18]:
# Specify the columns
columns = ['artist_id', 'artist_name']

# Initialize an empty DataFrame
df_artists_test = pd.DataFrame(columns={col: [] for col in columns})

count = 0
artist_set = set()

for _, row in test_df.iterrows():
    current_artist_id_list = eval(row["id_artists"])
    current_artist_name_list = eval(row["name_artists"])
    for cur_art_id, cur_art_name in zip(current_artist_id_list, current_artist_name_list):
        if cur_art_id not in artist_set:
            artist_set.add(cur_art_id)
            df_artists_test.loc[len(df_artists_test)] = [cur_art_id, cur_art_name]
            count += 1

print("amount of artists:", count)
df_artists_test

amount of artists: 13209


,artist_id,artist_name
0,5gLIbkXGeoJh0bzeEmvG2n,Underwater
1,1N477jyVWGcvVoRnvOePue,Gustavo Coutinho
2,3BL7Z4GDZQOEnlETaPYV93,Orgel Sound J-pop
3,3OpJxAHfT0KsJzQqdX5SLb,Ginger Tunes
4,59n7uqUFv9tprlcVU9CoXL,Jive Ass Sleepers
...,...,...
13204,69Mj3u4FTUrpyeGNSIaU6F,The J. Geils Band
13205,6piiChdjB4mmBVvOs7rWco,The Hollywood Stars
13206,3lJ6jvOiNY46L2juqXufpf,Changing Faces
13207,6JjTnHt1P7gQXlcFDoI4IC,Jellybean


In [19]:
import networkx as nx

all_artist_ids = artist_set.copy()

G2 = nx.Graph()

for id_art_cur in all_artist_ids:
    artist_name = df_artists_test[(df_artists_test["artist_id"] == id_art_cur)]["artist_name"]
    artist_info = df_artists_test[df_artists_test["artist_id"] == id_art_cur]
    if not artist_info.empty:  # Check if artist info exists
        # Filter out artists who are not successful
        node_attrs = {"name": artist_name.iloc[0]}
        G2.add_node(id_art_cur, **node_attrs)
    else:
        print("Artist info not found for ID:", id_art_cur)

for _, row in test_df.iterrows():
    artist_ids = list(eval(row["id_artists"]))
    artist_names = list(eval(row["name_artists"]))
    # Filter out songs that do not have more than one artist in their execution
    if len(artist_ids) > 1:
        for i in range(len(artist_ids)):
            for j in range(i+1, len(artist_ids)):
                artist_id_1 = str(artist_ids[i])
                lable1 = str(artist_names[i])
                artist_id_2 = str(artist_ids[j])
                lable2 = str(artist_names[j])
                # Check if the edge already exists
                if G2.has_edge(artist_id_1, artist_id_2):
                    # If the edge already exists, increment the weight by 1
                    G2[artist_id_1][artist_id_2]['weight'] += 2
                else:
                    # Add a new edge with weight 1
                    G2.add_edge(artist_id_1, artist_id_2, lable1=lable1, lable2=lable2, weight=2, song_id=str(row["song_id"]), song_name=str(row["song_name"]), explicit=bool(row["explicit"]), song_type=str(row["song_type"]), track_number=int(row["track_number"]), num_artists=int(row["num_artists"]), num_available_markets=int(row["num_available_markets"]), duration_ms=int(row["duration_ms"]), popularity=int(row["popularity"]), release_date=str(row["release_date"]), key = int(row["key"]), mode = int(row["mode"]), time_signature = int(row["time_signature"]), acousticness = float(row["acousticness"]), danceability = float(row["danceability"]), energy = float(row["energy"]), instrumentalness = float(row["instrumentalness"]), liveness = float(row["liveness"]),loudness = float(row["loudness"]), speechiness = float(row["speechiness"]), valence = float(row["valence"]), tempo = float(row["tempo"]))
    else:
        #prüfe ob die Artists Teil der Interessanten Menge sind
        artist_id_1 = str(artist_ids[0])
        lable1 = str(artist_names[0])
        if not G2.has_node(artist_id_1):
            node_attrs = {"name": lable1}
            G2.add_node(artist_id_1, **node_attrs)
        if G2.has_edge(artist_id_1, artist_id_1):
            # If the edge already exists, increment the weight by 1
            G2[artist_id_1][artist_id_1]['weight'] += 2
        else:
            G2.add_edge(artist_id_1, artist_id_1, lable1=lable1, lable2=lable1, weight=2, song_id=str(row["song_id"]), song_name=str(row["song_name"]), explicit=bool(row["explicit"]), song_type=str(row["song_type"]), track_number=int(row["track_number"]), num_artists=int(row["num_artists"]), num_available_markets=int(row["num_available_markets"]), duration_ms=int(row["duration_ms"]), popularity=int(row["popularity"]), release_date=str(row["release_date"]), key = int(row["key"]), mode = int(row["mode"]), time_signature = int(row["time_signature"]), acousticness = float(row["acousticness"]), danceability = float(row["danceability"]), energy = float(row["energy"]), instrumentalness = float(row["instrumentalness"]), liveness = float(row["liveness"]),loudness = float(row["loudness"]), speechiness = float(row["speechiness"]), valence = float(row["valence"]), tempo = float(row["tempo"]))

In [20]:
# Assuming G is your NetworkX graph

# Extract nodes and their attributes from the graph
nodes_with_attributes = [(G2.nodes[node], node) for node in G2.nodes()]

# Create a DataFrame from the nodes and their attributes
df_nodes_test = pd.DataFrame(nodes_with_attributes, columns=['Attributes', "Spotify ID"])

# Normalize the 'Attributes' column to expand it into separate columns
df_attributes = pd.json_normalize(df_nodes_test['Attributes'])

# Combine the original 'Node' column with the expanded attributes
df_nodes_test = pd.concat([df_attributes, df_nodes_test['Spotify ID']], axis=1)

# Display the DataFrame
df_nodes_test

,name,Spotify ID
0,Paul Vann,6OOyv8pNWEhHNPqnIZsHHw
1,Foghat,6x33CmZWo2Ve4hxYl2Craq
2,Josh Gannet,1j5ylyDXbkezoS849V9qLG
3,Mutemath,1J9Z9bhqkgyqseEi85cbyx
4,Patrick Jordan,4dLVmWvBrocIP03Dvj8W8c
...,...,...
13204,Big Rube,39uOfkRb9XnAEW9zs3JCbb
13205,The Tubes,7zfhej6FnVXN9LIXs6dcoK
13206,Cassius Jay,1d0f2TQJF6CgmJ9o52kH4x
13207,Normani,2cWZOOzeOm4WmBJRnD5R7I


In [21]:
df_nodes_test.to_csv("network_created/nodes_real_test.csv", sep='\t', index=False)

In [22]:
# Extract edges and their attributes from the graph
edges_with_attributes = [(u, v, G2.edges[u, v]) for u, v in G2.edges()]

# Create a DataFrame from the edges and their attributes
df_edges_test = pd.DataFrame(edges_with_attributes, columns=['Source', 'Target', 'Attributes'])

# Normalize the 'Attributes' column to expand it into separate columns
df_edge_attributes = pd.json_normalize(df_edges_test['Attributes'])

# Combine the original 'Attributes' column with the expanded attributes
df_edges_test = pd.concat([df_edges_test[['Source', 'Target']], df_edge_attributes], axis=1)
df_edges_test

,Source,Target,lable1,lable2,weight,song_id,song_name,explicit,song_type,track_number,...,time_signature,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,valence,tempo
0,6OOyv8pNWEhHNPqnIZsHHw,6OOyv8pNWEhHNPqnIZsHHw,Paul Vann,Paul Vann,26,7LCzpx2UwX71sRVPHU8xaO,A Nightingale Sang In Berkeley Square,False,Solo,1,...,4,0.53300,0.587,0.420,0.000000,0.0220,-9.123,0.0339,0.596,116.982
1,6x33CmZWo2Ve4hxYl2Craq,52yUzlU6QBN8NxqTXOWCnW,Foghat,Vinyl on HBO,2,4h5xW2wrCABWKXQwyHM4gI,I Just Want to Make Love to You,False,Collaboration,18,...,4,0.08770,0.387,0.711,0.000000,0.1950,-7.801,0.2360,0.569,123.223
2,6x33CmZWo2Ve4hxYl2Craq,6x33CmZWo2Ve4hxYl2Craq,Foghat,Foghat,96,0Q3NYWaj0oUTgowuzzTQuC,Honey Do List,False,Solo,10,...,4,0.16200,0.562,0.886,0.000001,0.1230,-5.482,0.1440,0.767,116.137
3,1j5ylyDXbkezoS849V9qLG,7xTKLpo7UCzXSnlH7fOIoM,Redman,Josh Gannet,4,0rUUAAVjrOvUZClNBpNdsE,Dr. Trevis (Intro) [feat. Josh Gannet],False,Collaboration,1,...,5,0.05380,0.672,0.473,0.000000,0.3010,-12.815,0.8890,0.665,125.008
4,1J9Z9bhqkgyqseEi85cbyx,3YQKmKGau1PzlVlkL1iodx,Twenty One Pilots,Mutemath,10,5dbgfhQpLiycbirhH3dJdh,Tear in My Heart (feat. MUTEMATH),False,Collaboration,4,...,4,0.02260,0.757,0.786,0.000030,0.1090,-6.090,0.0453,0.578,127.911
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16927,3VHwNUGuWO3pP7vHBYOYYe,3VHwNUGuWO3pP7vHBYOYYe,Francesco Demegni,Francesco Demegni,2,4tknKxqsZfNm4IYkFo5R0C,Simple Things,False,Solo,14,...,4,0.53100,0.750,0.448,0.915000,0.0925,-17.606,0.0433,0.730,120.048
16928,0h9smro0z3HqUbD94jotU8,0h9smro0z3HqUbD94jotU8,Smokey Robinson,Smokey Robinson,2,7tzogl9A0wN3eBrOdryfF2,Quiet Storm - Single Version,False,Solo,12,...,4,0.12700,0.180,0.636,0.000003,0.1570,-7.214,0.0530,0.400,175.508
16929,6Of5FvJNpTlTpfVCFisRKy,6Of5FvJNpTlTpfVCFisRKy,Timmy Vegas,Timmy Vegas,2,5ydCzy6vXBrRKd0Wb5zlKG,Get Yourself Together - Timmy's B'ham Disco Au...,False,Solo,10,...,4,0.00446,0.770,0.836,0.047200,0.0320,-4.339,0.0463,0.699,122.988
16930,0LthCv77K8NqLKr8B6266a,0LthCv77K8NqLKr8B6266a,Freak Nasty,Freak Nasty,66,7FU8V6XokeJ8iKn3o396bB,Shake Dat Thang,False,Solo,1,...,4,0.14100,0.922,0.460,0.002400,0.1450,-10.395,0.1160,0.573,128.043


In [23]:
# Save edge DataFrame to CSV
df_edges_test.to_csv('network_created/edges_real_test.csv', sep='\t', index=False)

In [24]:
num_nodes = G2.number_of_nodes()
num_edges = G2.number_of_edges()
print("Number of nodes:", num_nodes)
print("Number of edges:", num_edges)


Number of nodes: 13209
Number of edges: 16932


In [25]:
train_df

,song_id,song_name,id_artists,name_artists,popularity,explicit,song_type,track_number,num_artists,num_available_markets,...,danceability,energy,instrumentalness,liveness,loudness,speechiness,valence,tempo,hit,date
804676,7kXUEJmfvRXbzxOC0pHQgb,I Can't Believe I'm Losing You,['6eVCRmRG2HcfL4yx77A9WK'],['Jack Jones'],5,False,Solo,12,1,79,...,0.180,0.369,0.000000,0.1500,-9.797,0.0299,0.2240,81.701,0,1995-01-01
465447,1xdRFXv3ytSOdj8GuT3Rge,Thank You Lord (Interlude),['5NDMothbpdpq2xHqSjrrWn'],['Faith Evans'],20,False,Solo,10,1,77,...,0.370,0.284,0.000000,0.5580,-14.443,0.0378,0.6490,130.985,0,1995-01-01
465446,5wlQIBbKIWoB7AnSw09C6C,All This Love,['5NDMothbpdpq2xHqSjrrWn'],['Faith Evans'],26,False,Solo,9,1,77,...,0.605,0.604,0.000153,0.0693,-9.214,0.0480,0.5030,145.483,0,1995-01-01
377956,40uHAEjua1YmyKIJ6k0iVa,Bo Bo Wa,['3z6Uf5IZkN4TogRpRpgD84'],['Gary Wright'],0,False,Solo,8,1,79,...,0.481,0.652,0.008670,0.0397,-10.981,0.0364,0.7950,195.791,0,1995-01-01
465445,2zmMWWgwCXYarkO2SbyPZP,Come Over,['5NDMothbpdpq2xHqSjrrWn'],['Faith Evans'],38,False,Solo,7,1,77,...,0.534,0.563,0.000001,0.5740,-8.975,0.0601,0.4140,124.149,0,1995-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
677962,45oru9CcsrV3EWBMl1Wflg,Angels Moon,['7FQ8wLlE6kd6EBtexh0FAD'],['Richy Kicklighter'],0,False,Solo,6,1,79,...,0.580,0.480,0.907000,0.1100,-17.467,0.0317,0.6960,113.312,0,2015-08-19
819520,5AZQNFCdxA3W1nssrtu4Yr,能登半島 Originally Performed By 石川さゆり (オルゴール),['3BL7Z4GDZQOEnlETaPYV93'],['Orgel Sound J-pop'],0,False,Solo,7,1,79,...,0.434,0.498,0.923000,0.1310,-10.614,0.0572,0.0688,119.961,0,2015-08-19
678026,0gvKPCmhJzXa4YgptrNvI4,Up the Hill,['3FRfnhYD8uu3k8Dml7C7Wy'],['Jason Meeks'],0,False,Solo,30,1,79,...,0.813,0.645,0.826000,0.0773,-10.449,0.1390,0.4040,120.051,0,2015-08-19
819519,5FDxZkCXwOOO0CQP3zQUBL,浪花恋しぐれ Originally Performed By 都はるみ・岡千秋 (オルゴール),['3BL7Z4GDZQOEnlETaPYV93'],['Orgel Sound J-pop'],0,False,Solo,6,1,79,...,0.616,0.443,0.909000,0.0893,-9.730,0.0504,0.1430,140.022,0,2015-08-19


In [26]:
test_df

,song_id,song_name,id_artists,name_artists,popularity,explicit,song_type,track_number,num_artists,num_available_markets,...,danceability,energy,instrumentalness,liveness,loudness,speechiness,valence,tempo,hit,date
677960,0KkzGvp8twn623wysKZ4Th,Return to Ipanema,['5gLIbkXGeoJh0bzeEmvG2n'],['Underwater'],0,False,Solo,4,1,79,...,0.760,0.179,0.783000,0.1040,-21.624,0.0500,0.5980,138.898,0,2015-08-19
677959,6LYpAOMzSqyB2y2mNmqy5E,Foreign Feeling,['1N477jyVWGcvVoRnvOePue'],['Gustavo Coutinho'],0,False,Solo,3,1,79,...,0.430,0.274,0.786000,0.0887,-19.042,0.0307,0.2330,140.009,0,2015-08-19
819517,6EU7nvOyg2O8ld7UAKii7R,どうにもとまらない Originally Performed By 山本リンダ (オルゴール),['3BL7Z4GDZQOEnlETaPYV93'],['Orgel Sound J-pop'],0,False,Solo,4,1,79,...,0.595,0.594,0.919000,0.0993,-8.746,0.0551,0.1740,111.978,0,2015-08-19
677958,32bnbgvJ8FevA9rT2FxGAO,Wave,['3OpJxAHfT0KsJzQqdX5SLb'],['Ginger Tunes'],1,False,Solo,2,1,79,...,0.668,0.414,0.902000,0.1120,-10.019,0.0361,0.0399,121.084,0,2015-08-19
677957,7aMr9p35GaCmdWBmqbpfMa,Silver Samba,['59n7uqUFv9tprlcVU9CoXL'],['Jive Ass Sleepers'],0,False,Solo,1,1,79,...,0.692,0.487,0.070900,0.0860,-13.882,0.0324,0.7140,109.978,0,2015-08-19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11098,0xAqhyWvTkfcdFgm7dozMO,(I Wanna) Testify (Mono Single Version),['1Ve5PoI7i5jIIP1xudo6AB'],['The Parliaments'],7,False,Solo,10,1,79,...,0.668,0.702,0.000000,0.0521,-5.750,0.0330,0.7540,105.227,1,2019-08-23
10903,6zhg8TnVKi90ITDzFKwPr0,Swingtown - Alternate Version,['6QtGlUje9TIkLrgPZrESuk'],['Steve Miller Band'],26,False,Solo,5,1,79,...,0.459,0.764,0.000544,0.2440,-6.662,0.0354,0.4390,124.557,1,2019-08-23
5856,2GvkFSjRXV0Ii3uYnHiQ74,Take My Advice,['40PLSl99xXqAQsCQQPmfyM'],['Kym Sims'],0,False,Solo,10,1,73,...,0.737,0.814,0.392000,0.0844,-10.396,0.0307,0.8030,120.010,1,2019-08-23
5080,0lej8CemYKkWVHg77Xe1ky,A Man Holdin' on (To a Woman Lettin’ Go),"['1Fj0R2t4HaJa3oUe8azB8R', '5LDnSV99gbDSsMX3Dc...","['Ty Herndon', 'Anita Cochran']",14,False,Collaboration,7,2,79,...,0.611,0.429,0.000014,0.1120,-8.619,0.0298,0.2370,128.007,1,2019-08-23
